<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395 Redes Neuronales Artificiales 2019-1</h1>

<H3 align='center'> Tarea 1 - Redes Neuronales y *Deep Learning* </H3>
<hr style="height:2px;border:none"/>

## Integrantes

* _**Francisco Olivares - 201473575-8 - fracisco.olivars.14@sansano.usm.cl**_
* _**Gabriel Valenzuela - 201473505-7 - gabriel.valenzuel.14@sansano.usm.cl**_
* _**Felipe Vega - 201473511-1 - felipe.vega.14@sansano.usm.cl**_



<hr style="height:2px;border:none"/>

In [1]:
#Soy tu bloque pa' los imports papi
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact 


#Sklearn stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Keras stuff
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

#Que se vea ancho el jupyter
from IPython.display import IFrame
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


#Cantidad de columnas a mostrar con pandas
pd.set_option('display.max_columns',100)

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "c:\mauro\redesn~1\env\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    return importlib.import_module(mname)
  File "c:\mauro\redesn~1\env\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 978, in _gcd_import
  File "<frozen importlib._bootstrap>", line 961, in _find_and_load
  File "<frozen importlib._bootstrap>", line 950, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 648, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 560, in module_from_spec
  File "<frozen importlib._bootstrap_external>", line 922, in create_module
  File "<frozen importlib._bootstrap>", line 205, in _call_with_frames_removed
ImportError: DLL load failed: No se puede encontrar el módulo especificado.

During handling of the above exception, another exce

<hr style="height:2px;border:none"/>

# Red neuronal Feed Forward para detectar Exoplanetas

Para comenzar, se cargarán los datos necesarios, los cuales tienen features extraidas a partir de las curvas de luz (intensidad de la luz en el tiempo) y metadatos de la observación [continuar con el relleno].

Desde el archivo _**koi sets unb**_ se obtiene el conjunto al que pertenece cada dato, los cuales pueden ser Test, Train y Unlabeled. Desde _**koi light curves**_ se obtienen las features extraidas a partir de las curvas de luz y los metadatos de la observación. Finalmente, desde _**koi labels**_ se obtendrán las etiquetas de cada exoplaneta, las cuales son _**CONFIRMED**_ o _**FALSE POSITIVE**_.

In [ ]:
df_sets = pd.read_csv("data/koi_sets_unb.csv")

mask_train = (df_sets["Set"] == "Train").values
mask_test = (df_sets["Set"] == "Test").values

df_labels = pd.read_csv("data/koi_labels.csv")
df_X = pd.read_csv("data/koi_light_curves_X.csv")
df_labels_train = df_labels[mask_train]
df_labels_test = df_labels[mask_test]
df_X_train = df_X[mask_train]
df_X_test = df_X[mask_test]

Con los datos ya cargados, se generan las matrices para los conjuntos de entrenamiento y pruebas, como también se generan una etiqueta binaria para cada dato. Se realizar un boxplot para cada columna con datos numéricos, para observar la existencia de outliers.


In [ ]:
y_train = ((df_labels_train["NExScI Disposition"]=="CONFIRMED")*1).values
y_test = ((df_labels_test["NExScI Disposition"]=="CONFIRMED")*1).values
df_X_train = df_X_train.reset_index(drop=True)
df_X_test = df_X_test.reset_index(drop=True)

In [ ]:
def boxplotPlotTrain(column_name = 'Mean'):
    plt.figure()
    plt.title("Boxplot of "+column_name)
    plt.boxplot(df_X_train[column_name])
    plt.show()
    
numeric_columns = []
for i in df_X_train.columns:
    if np.issubdtype(df_X_train[i].dtype,np.number):
        numeric_columns.append(i)  
        
interact(boxplotPlotTrain, column_name=numeric_columns)        

Se observa la presencia de outliers en la gran mayoría de los datos numéricos. Dado lo anterior, se toma la decisión que si existen datos nulos se reemplazarán con la mediana, dado que es un estadístico robusto frente outliers. 

In [ ]:
df_X_train.fillna(df_X_train.median(), inplace=True)
df_X_test.fillna(df_X_test.median(), inplace=True)
X_train = df_X_train.values[:,1:]
X_test = df_X_test.values[:,1:]

> _**a) Explore los datos trabajados, ya sea con estadísticos simples o con gráficos como histogramas y/o boxplots. Comente sobre el problema enfrentado, es decir, la tarea de transformar un vector  𝑋  en un valor categórico (0 o 1).**_

El dataset cuenta con 41 atributos, cuyos nombres se presentan a continuación, 

In [ ]:
columns_names = df_X_train.columns[1:]
columns_names

In [ ]:
print(X_train.shape)
print(X_test.shape)


El conjunto de entrenamiento cuenta con 4692 registros, mientras que el conjunto de pruebas cuenta con 1565 registros.

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Number of examples on each label in Train Set")
plt.bar(['0','1'],[len(y_train)-y_train.sum(),y_train.sum()])
plt.xlabel("Labels")
plt.ylabel("Number of examples")
plt.xticks([0,1],['FALSE POSITIVE','CONFIRMED'])
plt.show()

Se observa que las etiquetas del conjunto de entrenamiento se encuentran desbalanceadas, por que se espera que la red tienda a clasificar cada registro como FALSE POSITIVE, pues eso minimizaría su probabilidad de equivocarse. Por lo anterior, al momento de evaluar el desempeño de la red se debe considerar además otra métrica distinta de la _accuracy_, como el _F1 Score_.

A continuación se presentan distintos estadísticos para cada uno de los atributos

In [ ]:
df_X_train.describe()

Se observan grandes diferencias entre el mínimo y el segundo cuartil, como también entre el máximo y el tercer cuartil, lo que concuerda con lo mostrado por los boxplots anteriores. También se ven las diferencias entre la media y la mediana, lo que refuerza la decisión de usar un estadístico robusto frente a outliers.

Se genera un histograma y un boxplot para cada una de las columnas en ambos dataset.

In [ ]:
def histAndBoxplot(column_name='Minimum', dataset='Train'):
    if(dataset=='Train'):
        df = df_X_train
    else:
        df = df_X_test
    plt.figure(figsize=(15,7))    
    plt.subplot(121)
    plt.title("Histogram of "+ column_name)
    plt.hist(df[column_name])
    plt.subplot(122)
    plt.title("Boxplot of " + column_name)
    plt.boxplot(df[column_name])
    plt.show()
    

In [ ]:
interact(histAndBoxplot,column_name = columns_names,dataset=['Train','Test'])

Se observa una gran cantidad de outliers en casi todos los atributos. Una posible decisión sería eliminar algunos de ellos, pero como no se tienen conocimientos astronómicos sobre Exoplanetas se prefiere mantenerlos en el dataset, pues podrían ser elementos específicos de cierto tipo de objetos astronómicos.

El problema a enfrentar es el de clasificación entre dos clases mutuamente excluyentes. Por lo tanto, a partir de un vector se debe obtener un valor categórico, lo que de utilizar Redes Neuronales Feed Forward se puede lograr mediante la aplicación de transformaciones entre capas de neuronas, donde finalmente la capa de salida cuente solo con dos neuronas, cuyo resultado de computo entregará la probabilidad de pertenecer a cierta etiqueta o no. Al ser etiquetaas mutuamente excluyentes se debe utilizar una función de activación adecuada, como por ejemplo la función Soft-Max, lo que entregará ambos resultados entre 0 o 1, donde además la suma entre ellos da 1. Finalmente, para entregar un valor entre 0 y 1, se determina cual de las probabilidades entregadas es mayor, y en función de eso se retorna la etiqueta de la clase requerida.

Cabe destacar que para resolver este tipo de problemas se busca aproximar $P(y|x)$, es decir, la pertenencia a una determinada clase dado un ejemplo en específico. Por lo anterior, se requiere utilizar una función de perdida adecuada en el entrenamiento, como por ejemplo la Cross-Entropy Loss, la cual calcula las divergencias entre la función de probabilidad que se desea aproximar y la aproximación. Acá surge un nuevo problema en este caso, pues al tener clases desbalanceadas el clasificador tenderá a clasificar un mayor cantidad de ejemplos con la etiqueta de la clase dominante, debido a que la Cross-Entropy Loss funciona en base a _*accuracy*_, y al clasificar todo con la etiqueta de la clase dominante este valor tendería a ser más alto si es desbalance es demasiado. [REVISAR EL CHAMUYO OE ZI]


> _**b) Escale los datos para ser trabajados por el modelo de aprendizaje, indique la importancia de éste paso. Además cree un conjunto de validación extrayendo un cierto porcentaje del conjunto de entrenamiento, por ejemplo el 20% manteniendo el desbalanceo de clases (split stratificado).**_

Se escalan los datos antes de trabajarlos con el modelo de aprendizaje, este paso es importante, pues al tener columnas con datos de distintas magnitudes, el modelo mayor importancia a aquellos atributos con magnitud más grande. En el caso de las Redes Neuronales Feed Forward, al momento de calcular la activación de cada capa. 

$$ a_i = \sigma\left( W_{i}^{T} \cdot a_{i-1} + b_{i}\right)$$

Al tener elementos con magnitudes más grandes afectarían directamente a las activaciones de las capas siguientes, por lo que un buen escalamiento de datos se hace realmente importante.

Además se separa un conjunto de validación, considerando un 20$\%$ de los datos del conjunto de entrenamiento, con el objetivo de usar dicho conjunto como un predictor del comportamiento en el conjunto de pruebas.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2,stratify=y_train,random_state=42)
print("Shape of X_train "+str(np.shape(X_train)))
print("Shape of X_val "+str(np.shape(X_val)))

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled =  scaler.transform(X_test)

> _**c) Muestre en un gráfico la función objetivo (cross entropy) para el conjunto de entrenamiento y de validación vs número de epochs de entrenamiento, para una red feedforward de 3 capas, con 256 unidades ocultas y función de activación sigmoidal. Entrene la red usando gradiente descendente estocástico con tasa de aprendizaje (learning rate) 0.01 y 100 epochs de entrenamiento. Comente. Si observa divergencia durante el entrenamiento, determine si esto ocurre para cada repetición del experimento. Compare el efecto de variar la función de activación a ReLU ¿Qué observa en la convergencia del modelo?**_

Se entrenan dos redes distintas, una con función de activación sigmoidal y la otra con función de activación Relu. Para cada red se grafica el valor de su Loss Function en cada Epoch para los conjuntos de entrenamiento y validación. Como se está utilizando gradiente descendente estocástico, se repite el experiento 25 veces.

In [ ]:
hist_list = []
hist_list_relu = []
epoch_list = [x for x in range(1,101)]
for i in range(25):
    model = Sequential()
    model.add(Dense(256, input_dim=X_train_scaled.shape[1],kernel_initializer='uniform',activation='sigmoid'))
    model.add(Dense(1, kernel_initializer='uniform',activation="sigmoid"))
    model.compile(optimizer=SGD(lr=0.01),loss='binary_crossentropy')
    hist_list.append(model.fit(X_train_scaled,y_train,epochs=100,verbose=0,validation_data=(X_val_scaled,y_val)))
  
    model = Sequential()
    model.add(Dense(256, input_dim=X_train_scaled.shape[1],kernel_initializer='uniform',activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform',activation="relu"))
    model.compile(optimizer=SGD(lr=0.01),loss='binary_crossentropy')
    hist_list_relu.append(model.fit(X_train_scaled,y_train,epochs=100,verbose=0,validation_data=(X_val_scaled,y_val)))

In [ ]:
plt.figure(figsize=(20,7))
plt.subplot(121)
plt.title('Cross-Entropy Loss versus Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')


for i in hist_list:
    plt.plot(epoch_list,i.history['loss'],color='blue')
    plt.plot(epoch_list,i.history['val_loss'],color='green')

plt.legend(['Train Sigmoid','Validation Sigmoid'],loc='upper right')
  
plt.subplot(122)
plt.title('Cross-Entropy Loss versus Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')

for j in hist_list_relu:
    plt.plot(epoch_list,j.history['loss'],color='red')
    plt.plot(epoch_list,j.history['val_loss'],color='yellow')

plt.legend(['Train Relu','Validation Relu'],loc='upper right')
plt.show()  


En el caso de las redes con activación sigmoidal, en los 25 experimentos se observa convergencia en el entrenamiento, además no se observa overfitting. En el caso de las redes con activacion ReLu, se observa divergencia en gran parte de los experimentos. Esto probablemente se deba a que la función rectificadora no está acotada por la derecha, por lo que es probable que eso afecte en las predicciones del modelo mientras se entrena.

> _**Finalmente compare a través de una métrica de desempeño sobre el conjunto de pruebas, en este caso como trabajamos un problema desbalanceado, mida f1 score weighted, comente sobre esta decisión ¿Es esperable la diferencia entre relu y sigmoidal en base a los gráficos realizados?**_

Para evaluar ambas redes se utiliza el f1 score, el cual trata de considerar la _**accuracy**_ y el _**recall**_. La precisión, en un problema de clasificación binario, se puede ver como la razón entre los verdaderos positivos y todos los valores entregados como positivos por el modelo. El recall en cambio, se puede ver como la razón entre los verdaderos positivos entre todos los valores verdaderos del conjunto. En problemas desbalanceados, si se considera solo la accuracy para evaluar los distintos modelos, se tenderá a elegir un modelo que clasifique la gran mayoría de los datos con los valores de la clase dominante, en cambio si se utiliza solo el recall para elegir el modelo, dicho modelo tenderá a clasificar todos los catos como la clase menos representada. Es por lo anterior que se utiliza el F1-Score, pues considera la media armónica entre ambas métricas.

$$ F1_{score} = 2\cdot\frac{precision \cdot recall}{precision + recall}$$

Considerando los gráficos anteriores de ambas redes, es de esperar que la red con activación sigmoidal obtenga mejores resultados. Pues en el caso de la red con activación Relu, si se observa que el valor de la Loss es muy alto es porque $P(y|x)$ y $f(x)$ divergen mucho entre si.